元编程将程序当做数据，或在运行期完成编译期的工作。

装饰器类似于AOP编程范式，即面向切面编程，就是在不修改目标源码的前提下，添加功能的技术手段或设计模式，它是对OOP的补充。

In [1]:
def log(fn):
    def wraps(*args, **kwargs):
        print(*args, **kwargs)
        return fn(*args, **kwargs)
    return wraps

In [2]:
import dis

In [3]:
dis.dis(log)

  2           0 LOAD_CLOSURE             0 (fn)
              2 BUILD_TUPLE              1
              4 LOAD_CONST               1 (<code object wraps at 0x1079fc270, file "<ipython-input-1-bc3d2593a215>", line 2>)
              6 LOAD_CONST               2 ('log.<locals>.wraps')
              8 MAKE_FUNCTION            8
             10 STORE_FAST               1 (wraps)

  5          12 LOAD_FAST                1 (wraps)
             14 RETURN_VALUE


In [4]:
def add(x,y):
    return x+y

In [7]:
log(add)(1,4)

1 4


5

In [8]:
@log
def add(x,y):
    return x+y

In [9]:
dis.dis(add)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (args)
              4 LOAD_FAST                1 (kwargs)
              6 CALL_FUNCTION_EX         1
              8 POP_TOP

  4          10 LOAD_DEREF               0 (fn)
             12 LOAD_FAST                0 (args)
             14 LOAD_FAST                1 (kwargs)
             16 CALL_FUNCTION_EX         1
             18 RETURN_VALUE


In [10]:
def log(param):
    def decorator(fn):
        def wraps(*args, **kwargs):
            print(*args, **kwargs)
            return fn(*args, **kwargs)
        return wraps
    return decorator

In [11]:
dis.dis(log)

  2           0 LOAD_CONST               1 (<code object decorator at 0x107b13c90, file "<ipython-input-10-9f974b1647fa>", line 2>)
              2 LOAD_CONST               2 ('log.<locals>.decorator')
              4 MAKE_FUNCTION            0
              6 STORE_FAST               1 (decorator)

  7           8 LOAD_FAST                1 (decorator)
             10 RETURN_VALUE


In [12]:
@log('haha')
def add(x,y):
    return x+y

In [13]:
add(1,4)

1 4


5

In [16]:
def add(x,y):
    return x+y
log('haha')(add)(1,4)

1 4


5

类实现的装饰器用于实例方法时，会导致方法绑定丢失。

In [17]:
class log:
    def __init__(self, fn):
        self.fn = fn
    def __call__(self, *args, **kwargs):
        print('log ',*args, **kwargs)
        return self.fn(*args, **kwargs)

In [23]:
class X:
    @log
    def test(self):
        print(1+2)
        return 5

In [27]:
x = X()

In [20]:
x.test  # 方法被装饰器实例替代

In [21]:
x.test()

log 


TypeError: test() missing 1 required positional argument: 'self'

In [28]:
x.test(x)

log  <__main__.X object at 0x107b3c5c0>
3


5

因为装饰器实例替换了方法，结果导致实现绑定的描述符方法被隐藏，无法自动调用。要么让装饰器也实现描述符协议，要么显式传递参数。这样反而不如函数简单，因为函数默认实现了描述符协议和绑定规则，状态维持的话函数一样可以添加属性

多个装饰器嵌套时，我们需确保类型方法的装饰器是最外面的一个，因为无法确定内层装饰器的实现，可能也会因为描述符问题导致方法绑定失效。

In [ ]:
import functools
def log(fn):
    @functools.wraps(fn)
    def wrapped(*args, **kwargs):
        print(f'wrap:{id(wrapped)}, fn:{id(fn)}')
        return fn(*args, **kwargs)
    return wrapped

In [33]:
@log
def add(x:int, y:int) -> int:
    return x+y

In [34]:
add(1,6)

wrap:4424115200, fn:4424114792


7

In [35]:
add.__name__

'add'

In [36]:
add.__annotations__

{'return': int, 'x': int, 'y': int}

In [37]:
id(add)

4424115200

In [39]:
id(add.__wrapped__)

4424114792

functools.wraps会将原函数的\_\_module__, \_\_name__, \_\_doc__, \_\_annotations__等属性复制到包装函数，然后将原函数或上一装饰器保存在\_\_wrapped__中

In [41]:
def log(cls):
    
    class wrapper:
        def __init__(self, *args, **kwargs):
            self.__dict__['inst'] = cls(*args, **kwargs)
        def __getattr__(self, name):
            value = getattr(self.inst, name)
            print('get value ', value)
            return value
        def __setattr__(self, name, value):
            print('set value', value)
            return setattr(self.inst, name, value)
    return wrapper

In [42]:
@log
class X:
    pass

In [43]:
x = X()

In [44]:
x.a = 5

set value 5


In [45]:
x.a

get value  5


5

用于类别的装饰器，也可以用函数来写

In [46]:
def log(cls):
    def wrapper(*args, **kwargs):
        o = cls(*args, **kwargs)
        print('log ',o)
        return o
    return wrapper
@log
class X:
    pass

In [47]:
X()

log  <__main__.X object at 0x107b461d0>
